In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import tensorflow.keras.layers as layers

In [ ]:
def f(x):
    return 30*np.log(x[:,0]) - 0.01*x[:,0] + 0.01*x[:,1]
nb_points = 1_000_000

In [ ]:
X = np.random.randint(low=1,high=10**3,size=(nb_points,2))
Y = f(X)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(X[:10_000,0], X[:10_000,1], Y[:10_000])

In [ ]:
X_train, Y_train = X[:int(nb_points*0.8)], Y[:int(nb_points*0.8)]
X_valid, Y_valid = X[int(nb_points*0.8):], Y[int(nb_points*0.8):]

In [ ]:
inputs = layers.Input(shape=(2,))
x = layers.Dense(128, activation='relu')(inputs)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(1, activation='relu')(x)

model = tf.keras.Model(inputs, outputs, name="function_guesser")

In [ ]:
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss=tf.losses.mean_squared_error, metrics=[tf.losses.mean_absolute_error])
model.summary()

In [ ]:
history = model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_data=(X_valid, Y_valid))

In [ ]:
plt.plot(history.history['mean_absolute_error'], label="train error")
plt.plot(history.history['val_mean_absolute_error'], label="valid error")
plt.legend()

In [ ]:
random_point = (5,17)
random_point = np.array(random_point).reshape((1,2))
print(model.predict(random_point), "vs", f(random_point))

In [ ]:
Y_pred = model.predict(X[:10000])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(X[:10_000,0], X[:10_000,1], Y_pred[:10_000])